In [1]:
import pandas as pd
import difflib

In [2]:
dan = pd.read_csv("NetflixViewingHistoryDan.csv")
netflix = pd.read_csv("netflix_titles.csv")

In [3]:
bestMatches = dan['Title'].apply(lambda x: difflib.get_close_matches(x, netflix['title']))
bestMatches.head(10)

0                  [Big Time Rush, Big Time, Big Kill]
1    [Monster Island, Monster House, Super Monsters...
2                                                   []
3                            [Brand New Cherry Flavor]
4    [Starship Troopers: Traitor of Mars, Star Trek...
5         [I Think You Should Leave with Tim Robinson]
6                                                   []
7         [I Think You Should Leave with Tim Robinson]
8                                                   []
9                                                   []
Name: Title, dtype: object

In [4]:
genres = []
for i in range(len(bestMatches)):
    itemGenres = []
    for j in range(len(bestMatches[i])):
        itemGenres.append(netflix['listed_in'].values[netflix['title'] == bestMatches[i][j]])
    genres.append(itemGenres)
genres[:10]

[[array(["Kids' TV"], dtype=object),
  array(['Documentaries'], dtype=object),
  array(['Action & Adventure'], dtype=object)],
 [array(['Children & Family Movies'], dtype=object),
  array(['Children & Family Movies, Comedies'], dtype=object),
  array(['Children & Family Movies'], dtype=object)],
 [],
 [array(['TV Dramas, TV Horror, TV Mysteries'], dtype=object)],
 [array(['Action & Adventure, Sci-Fi & Fantasy'], dtype=object),
  array(['Action & Adventure, Sci-Fi & Fantasy'], dtype=object),
  array(["Kids' TV"], dtype=object)],
 [array(['TV Comedies'], dtype=object)],
 [],
 [array(['TV Comedies'], dtype=object)],
 [],
 []]

In [5]:
finalGenres = []
for i in range(len(genres)):
    for j in range(len(genres[i])):
        for k in range(len(genres[i][j])):
            finalGenres += list(genres[i][j])

finalGenres[:10]

["Kids' TV",
 'Documentaries',
 'Action & Adventure',
 'Children & Family Movies',
 'Children & Family Movies, Comedies',
 'Children & Family Movies',
 'TV Dramas, TV Horror, TV Mysteries',
 'Action & Adventure, Sci-Fi & Fantasy',
 'Action & Adventure, Sci-Fi & Fantasy',
 "Kids' TV"]

In [6]:
genreCounts = pd.Series(finalGenres).value_counts()
genreCounts[:10]

Stand-Up Comedy                       88
Action & Adventure                    48
TV Comedies                           39
Docuseries                            34
British TV Shows, Reality TV          33
Dramas                                22
Documentaries                         21
Action & Adventure, Comedies          20
Dramas, International Movies          19
Children & Family Movies, Comedies    14
dtype: int64

In [7]:
listOfGenres = list(genreCounts.keys())
listOfGenres[:10]

['Stand-Up Comedy',
 'Action & Adventure',
 'TV Comedies',
 'Docuseries',
 'British TV Shows, Reality TV',
 'Dramas',
 'Documentaries',
 'Action & Adventure, Comedies',
 'Dramas, International Movies',
 'Children & Family Movies, Comedies']

In [8]:
genresAndMovies = {}
for i in range(netflix.shape[0]):
    if not genresAndMovies.get(netflix.iloc[i]['listed_in']):
        genresAndMovies[netflix.iloc[i]['listed_in']] = []
    genresAndMovies[netflix.iloc[i]['listed_in']].append(netflix.iloc[i]['title'])
#genresAndMovies

In [9]:
closeMovies = {}
for i in range(10):
    key = listOfGenres[i]
    movies = genresAndMovies.get(key)
    closeMovies[key] = []
    listOfLists = list(dan["Title"].apply(lambda x: difflib.get_close_matches(x, movies)))
    for subList in listOfLists:
        closeMovies[key] += subList


In [10]:
for key, value in closeMovies.items():
    print(key)
    print(list(pd.Series(value).value_counts().keys())[:20])
    print()

Stand-Up Comedy
['Jim Gaffigan: Obsessed', 'Jim Gaffigan: Cinco', 'Kevin Hart: Laugh at My Pain', 'Jim Jefferies: Intolerant', 'Jim Jefferies: Freedumb', 'Jim Gaffigan: King Baby', 'Hannibal Buress: Comedy Camisado', 'Trevor Noah: Son of Patricia', 'Patton Oswalt: Talking for Clapping', 'Jim Jefferies : BARE', 'Tom Segura: Completely Normal', 'Tom Segura: Mostly Stories', 'Bert Kreischer: Hey Big Boy', 'Trevor Noah: Afraid of the Dark', 'Kevin Hart: What Now?', 'Kevin Hart: Let Me Explain', 'Jim Gaffigan: Mr. Universe', 'Patton Oswalt: Annihilation', 'Tom Segura: Ball Hog', 'Bert Kreischer: The Machine']

Action & Adventure
['The Losers', 'The Saint', 'The Prince', 'The Art of War', 'The Peacemaker', 'Ninja Assassin', 'American Assassin', 'Kill Bill: Vol. 2', 'Olympus Has Fallen', 'Angel Has Fallen', 'Act of Valor', 'The Old Guard', 'The Hard Way', 'The Long Riders', 'The 2nd', 'Kill Bill: Vol. 1', 'Takers', 'Casino Royale', 'Pusher', 'Troy: The Odyssey']

TV Comedies
['The Office (U.S